# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

### Step 1: Create Data Owners

In [3]:
!pip install syft

     |████████████████████████████████| 225kB 3.4MB/s 
     |████████████████████████████████| 450kB 58.3MB/s 
     |████████████████████████████████| 81kB 29.0MB/s 
     |████████████████████████████████| 204kB 60.2MB/s 
     |████████████████████████████████| 389kB 36.6MB/s 
     |████████████████████████████████| 1.4MB 49.8MB/s 
     |████████████████████████████████| 256kB 48.8MB/s 
     |████████████████████████████████| 51kB 27.2MB/s 
     |████████████████████████████████| 276kB 63.0MB/s 
     |████████████████████████████████| 122kB 59.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0709 23:15:12.400654 140450114865024 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0709 23:15:12.423265 140450114865024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0710 00:30:25.024554 140147598481280 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:30:25.027043 140147598481280 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:30:25.028194 140147598481280 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:30:25.029717 140147598481280 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:30:25.031599 140147598481280 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:30:25.032658 140147598481280 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


### Step 2: Create the Model

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [0]:
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)


### Step 4: Train Bob's and Alice's Models (in parallel)

In [0]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')


Bob loss 0.27044060826301575
alice loss 1.5197663307189941
Bob loss 0.15266671776771545
alice loss 0.5376052856445312
Bob loss 0.08809097111225128
alice loss 0.23955757915973663
Bob loss 0.052534446120262146
alice loss 0.14449623227119446
Bob loss 0.03281897306442261
alice loss 0.11006404459476471
Bob loss 0.021761268377304077
alice loss 0.09410572052001953
Bob loss 0.015445023775100708
alice loss 0.08410660922527313
Bob loss 0.011734331957995892
alice loss 0.07632281631231308
Bob loss 0.009463630616664886
alice loss 0.0696081593632698
Bob loss 0.00799625925719738
alice loss 0.0635882094502449


### Step 5: Send Both Updated Models to a Secure Worker

In [0]:
alices_model.move(secure_worker) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(secure_worker)


In [0]:
secure_worker._objects # can you identify those objects in the Secure worker?

{9649689706: Parameter containing:
 tensor([[-0.6767,  0.1668]], requires_grad=True),
 34241585867: Parameter containing:
 tensor([-0.0781], requires_grad=True),
 59561137517: Parameter containing:
 tensor([[-0.0912,  0.4779]], requires_grad=True),
 74227018814: Parameter containing:
 tensor([0.8201], requires_grad=True)}

### Step 6: Average The Models

In [0]:
# A pytorch trick: to set the weights: you have to either access the data[idx] 
# or wrap the set function with (with torch.no_grad():)
# below we illustrate the first way
# the next cell demonstrates the complete way:

model.weight.data[0].set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data[0].set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

tensor([0.3710])

### Step 7: The complete solution over multiple training sessions

In [0]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

NameError: ignored

In [0]:
preds = model(data)
loss = ((preds - target) ** 2).mean()

In [0]:
print(preds)
print(target)
print(loss.data)

tensor([[0.1812],
        [0.1568],
        [0.8002],
        [0.7759]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0369)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [0]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted ( you are the dealer, thus still can see --and know everything-- but Alice and Bob did not really know anything about the overall deal --the sum of the numbers; the secrete--)

In this case, however, to keep the secrete is not about the relaiability / integrity of the person who holds it, rather the fact that those shareholders do not know each other!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant.

In [0]:
x = 5

Q = 9923 # the little fix; a very large prime number

bob_x_share = 1525 # give bob a random number.. try with negatives

alice_x_share = Q - bob_x_share + x

print(f'Bob share= {bob_x_share}')
print(f'Alice share= {alice_x_share}')
print(f'Add both shares= {bob_x_share + alice_x_share}... Wrong secrete')

Bob share= 1525
Alice share= 8403
Add both shares= 9928... Wrong secrete


In [0]:
# Decryption happens by summing all the shares together MODULUS some constant

(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so:  `x_share = (2,5,7)`

Even though normally those shares would be distributed among several workers, you can store them in ordered tuples like this for now.

In [0]:
import random

Q = 23740629843760239486723

# accepts a secrete and shares it among n_shahre, returns a tuple with n_share shares
def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

  # accepts a tuple of shares and return the decrypted values
def decrypt(shares):
    return sum(shares) % Q

In [0]:
shares = encrypt(x=7, n_share=10)
shares

(22479832612412023704979,
 8302679796681329496055,
 3372711692182172481529,
 9921427517126808100478,
 11018003054531615741608,
 1665892409661174510477,
 1657124149977544149884,
 7758834287806279635999,
 11295600847989464095597,
 17490413006672546030293)

In [0]:
decrypt(shares)

7

In [0]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
# run this block of code multiple times, what do you notice baout x and y?

x = encrypt(5)
print(x)
y = encrypt(7)
print(y)

z = add(x,y)
decrypt(z)

(15120723338006739353608, 23287994932254469990443, 9072541417259269629400)
(22302551296749638383904, 4362569530253349207143, 20816138860517491382406)


12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE= 10  # integer numbers (normal numbers)
PRECISION= 3  # how many porecisions after the decimal point to encode
Q = 23740629843760239486723 

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(-0.600)

23740629843760239345664

In [0]:
decode(23740629843760239345664)

-141.059

In [0]:
# run this cell multiple times and notice the changes

x = encrypt(encode(5.5))
y = encrypt(encode(5.5))
z = add(x,y)


print(f'Value of x = {x}')
print(f'Value of y = {y}')
print(f'Value of sum = {z}')
print(f'decoded sum = {decode(decrypt(z))}')

Value of x = (2119933375207712057588, 18902700362692559143737, 2717996105859968290898)
Value of y = (17248568464150894911686, 1976282359815612015543, 4515779019793732564994)
Value of sum = (19368501839358606969274, 20878982722508171159280, 7233775125653700855892)
decoded sum = 11.0


# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
# Its a good idea to restrat the environment at this point

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")


bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0709 23:41:50.837670 140450114865024 hook.py:98] Torch was already hooked... skipping hooking process
W0709 23:41:50.840801 140450114865024 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:41:50.844214 140450114865024 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:41:50.851465 140450114865024 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:41:50.853125 140450114865024 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:41:50.854395 140450114865024 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:41:50.856119 140450114865024 base.py:628] Worker bob already exists. Replacing 

<VirtualWorker id:secure_worker #objects:44>

In [0]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

### Secret Sharing Using PySyft

We can share using the simple` .share()` method!

In [0]:
x = x.share(bob, alice, ted) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:51921514687 -> bob:45786540873]
	-> (Wrapper)>[PointerTensor | me:21219884382 -> alice:94480426237]
	-> (Wrapper)>[PointerTensor | me:56362002564 -> ted:27598738353]
	*crypto provider: me*

In [0]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{6890318660: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 12720685973: tensor([[0.],
         [0.]], requires_grad=True), 45786540873: tensor([2639283518613220116, 2837107656649447762, 3570810798842993920,
         1622579968323960749, 2171830530492123658]), 95427661298: tensor([[ 0.0560],
         [-0.0188]], grad_fn=<AddmmBackward>)}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:86557190265 -> bob:52765790824]
	-> (Wrapper)>[PointerTensor | me:27866052863 -> alice:48304828768]
	-> (Wrapper)>[PointerTensor | me:76248298544 -> ted:31067998901]
	*crypto provider: me*

In [0]:
bob._objects # bob has another tensor.. why?

{6890318660: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 12720685973: tensor([[0.],
         [0.]], requires_grad=True), 45786540873: tensor([2639283518613220116, 2837107656649447762, 3570810798842993920,
         1622579968323960749, 2171830530492123658]), 52765790824: tensor([5278567037226440232, 5674215313298895524, 7141621597685987840,
         3245159936647921498, 4343661060984247316]), 95427661298: tensor([[ 0.0560],
         [-0.0188]], grad_fn=<AddmmBackward>)}

In [0]:
y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])


### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using `.fix_precision()`
This is important when doing FL since the values you are sharing are decimal numbers.

**Q: what values are we sharing? what are the shared secretes?**

In [0]:
x = th.tensor([0.1, 0.2, 0.3, 0.4]) # a decimal tensor!

In [0]:
x

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [0]:
x = x.fix_prec() # call this function to encode the decimal tensor using fixed precision
x # notice that this is a tensor chain <-------

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400])

In [0]:
print(f'Type of x {type(x)}')
print(f'Type of x.child {type(x.child)}')
print(f'Type of x.child.child {type(x.child.child)}') # <-- this is how you get the data from a fixed precision tensor

Type of x <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>
Type of x.child <class 'syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor'>
Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [0]:
# to decode the previous tensor
print(x.float_prec())

tensor([0.1000, 0.2000, 0.3000, 0.4000])


In [0]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600, 800])

In [0]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:1597106775 -> bob:51515872593]
	-> (Wrapper)>[PointerTensor | me:8713264662 -> alice:46919393944]
	*crypto provider: me*

In [0]:
bob._objects

{6890318660: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 12720685973: tensor([[0.],
         [0.]], requires_grad=True), 45786540873: tensor([2639283518613220116, 2837107656649447762, 3570810798842993920,
         1622579968323960749, 2171830530492123658]), 51515872593: tensor([2536254723988119534, 1421534363243017994, 1934572373662793766]), 95427661298: tensor([[ 0.0560],
         [-0.0188]], grad_fn=<AddmmBackward>)}

In [0]:
y = x + x

In [0]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Reuse your project from the Secuered Agggregator above to train the same model using the FL appraoch, but this time use the Additive Sharing Encryption so that the participating members have access to their own models only. There is no secure aggregator in this senario. 
Include four members in this project (Alice, Bob, Ted, and Carol). 

Hint: we will *share* the model with the shareholders, do addition on the encrypted models, and then aggregate the final model locally. 

In [1]:
import syft as sy
import torch as th
from torch import nn, optim
import torch.nn.functional as F

hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="carol")
carol = sy.VirtualWorker(hook, id="carol")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")
carol = sy.VirtualWorker(hook, id="carol")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")


# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, ted, carol,crypto_provider])
alice.add_workers([bob, ted, carol,crypto_provider])
ted.add_workers([alice, bob, carol,crypto_provider])
carol.add_workers([alice, bob, ted,crypto_provider])
crypto_provider.add_workers([alice, bob, ted,carol])

W0712 00:00:46.816105 140387329456000 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0712 00:00:46.831608 140387329456000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0712 00:00:47.718270 140387329456000 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0712 00:00:47.719422 140387329456000 base.py:628] Worker ted already exists. Replacing old worker which could cause                     unexpected behavior
W0712 00:00:47.720657 140387329456000 base.py:628] Worker carol already exists. Replacing old worker which could cause   

<VirtualWorker id:crypto_provider #objects:0>

In [2]:
model = nn.Linear(2,1)
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = th.tensor([[0],[0],[1],[1.]])
data = data.fix_precision().share(bob, alice, crypto_provider=crypto_provider, requires_grad=True)#using crypto_provider because bob couldn't find local
target = target.fix_precision().share(bob, alice, crypto_provider=crypto_provider, requires_grad=True)
model = model.fix_precision().share(bob, alice,crypto_provider=crypto_provider, requires_grad=True)
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()
for iter in range(20):
    opt.zero_grad()
    pred = model(data)
    loss = ((pred - target)**2).sum()
    loss.backward()
    opt.step()
    print(loss.get().float_precision())
#model.get().float_precision()
#print(model.weight.data)
#print(model.bias.data)

tensor([1.4380])
tensor([0.8440])
tensor([0.5360])
tensor([0.3430])
tensor([0.2210])
tensor([0.1430])
tensor([0.0910])
tensor([0.0590])
tensor([0.0380])
tensor([0.0250])
tensor([0.0160])
tensor([0.0100])
tensor([0.0070])
tensor([0.0030])
tensor([0.0020])
tensor([0.0040])
tensor([0.0020])
tensor([0.0020])
tensor([0.0020])
tensor([0.])
